In [ ]:
import pandas as pd
import yfinance as yf
import os

# Step 1: Load Company and Sector Information
nifty500_df = pd.read_csv('ind_nifty500list.csv')
companies = nifty500_df['Symbol'].tolist()
sectors = nifty500_df.set_index('Symbol')['Industry'].to_dict()

# Create a directory to save data if it doesn't exist
os.makedirs('stock_data', exist_ok=True)

# Step 2: Scrape Historical Stock Data and Compute Features
def fetch_and_process_data(ticker):
    try:
        stock = yf.download(ticker + ".NS", start="2020-01-10", end="2025-01-10")
        if stock.empty:
            print(f"No data for {ticker}")
            return
        
        # Calculate Moving Averages
        stock['MA_20'] = stock['Close'].rolling(window=20).mean()
        stock['MA_50'] = stock['Close'].rolling(window=50).mean()
        
        # Calculate Volatility (Standard Deviation)
        stock['Volatility_20'] = stock['Close'].rolling(window=20).std()
        stock['Volatility_50'] = stock['Close'].rolling(window=50).std()
        
        # Save to CSV
        stock.to_csv(f'stock_data/{ticker}_data.csv')
        print(f"Data for {ticker} saved.")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

for ticker in companies:
    fetch_and_process_data(ticker)

print("Data scraping and feature computation completed.")


In [3]:
pip install yfinance

  Attempting uninstall: requests
    Found existing installation: requests 2.29.0
    Uninstalling requests-2.29.0:
      Successfully uninstalled requests-2.29.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires importlib-metadata<8,>=1.4, but you have importlib-metadata 8.4.0 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
chromadb 0.5.3 requires tenacity>=8.2.3, but you have tenacity 8.2.2 which is incompatible.
unstructured-client 0.23.8 requires certifi>=2023.7.22, but you have certifi 2022.12.7 which is incompatible.
unstructured-client 0.23.8 requires charset-normalizer>=3.2.0, but you have charset-normalizer 3.1.0 which is incompatible.
unstructured-client 0.23.8 requires urllib3>=1.26.18, but you have urllib3 1.26.15 which is incompatible.
Note: you may nee

In [35]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import yfinance as yf
import os

# Step 1: Load Company and Sector Information
def load_company_data(filepath):
    df = pd.read_csv(filepath)

    label_encoder = LabelEncoder()
    df['Industry Encoded']= label_encoder.fit_transform(df['Industry'])
    return df['Symbol'].tolist(), df['Industry'].tolist(), df['Industry Encoded'].tolist()

# Step 2: Fetch Stock Data
def fetch_stock_data(ticker, start_date, end_date):
    try:
        data = yf.download(f"{ticker}.NS", start=start_date, end=end_date)
        return data if not data.empty else None
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

from sklearn.preprocessing import StandardScaler

# Step 3: Compute Financial Features
def compute_features(stock_data):
    stock_data['MA_20'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['MA_50'] = stock_data['Close'].rolling(window=50).mean()
    
    scaler = StandardScaler()
    stock_data['Normalised Close'] = scaler.fit_transform(stock_data[['Close']])
    # print(stock_data['Normalised Close'])
    stock_data['Return Ratio'] = (stock_data['Close']-stock_data['Close'].shift(1))/stock_data['Close'].shift(1)
    
    stock_data['Percentage Change Open'] = stock_data['Open']/stock_data['Close'] - 1
    stock_data['Percentage Change High'] = stock_data['High']/stock_data['Close'] - 1
    stock_data['Percentage Change Low'] = stock_data['Low']/stock_data['Close'] - 1


    return stock_data

# Step 4: Align Data with Reference Dates
def align_with_reference(stock_data, reference_dates):
    print(f"Reference Dates: {reference_dates} Current Date Range: {stock_data.index[0]} to {stock_data.index[-1]} check is {stock_data.index[0] != reference_dates[0]}")
    if stock_data.index[0] > pd.Timestamp("2022-01-10"):
        return False
    else:
        return True
    

# Step 5: Main Execution
def main():
    start_date, end_date = "2020-01-10", "2025-01-10"
    companies, sector, sector_encoded  = load_company_data('ind_nifty500list.csv')
    os.makedirs('stock_data', exist_ok=True)
    
    reference_data = fetch_stock_data("ONGC", start_date, end_date)
    if reference_data is None:
        print("Failed to fetch reference data.")
        return
    reference_dates = (reference_data.index[0], reference_data.index[-1])
    missing = []
    for ticker in companies:
        stock_data = fetch_stock_data(ticker, start_date, end_date)
        if stock_data is not None:
            if not align_with_reference(stock_data, reference_dates):
                print("Skipping..."+ticker)
                missing.append(ticker)
                continue
            stock_data = compute_features(stock_data)
            # stock_data.fillna(0, inplace=True)
            stock_data['Sector'] = sector[companies.index(ticker)]
            stock_data['Sector Encoded'] = sector_encoded[companies.index(ticker)]
            stock_data.to_csv(f'stock_data/{ticker}_data.csv')
            print(f"Processed and saved data for {ticker}.")
        else:
            print(f"Skipping {ticker} due to missing data.")
            missing.append(ticker)
    with open("missing_nw.txt", "w") as file:
        for line in missing:
            file.write(line + "\n")
    print("All data processing completed.")

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Reference Dates: (Timestamp('2020-01-10 00:00:00'), Timestamp('2025-01-09 00:00:00')) Current Date Range: 2020-01-10 00:00:00 to 2025-01-09 00:00:00 check is False
Processed and saved data for 360ONE.


[*********************100%***********************]  1 of 1 completed


Reference Dates: (Timestamp('2020-01-10 00:00:00'), Timestamp('2025-01-09 00:00:00')) Current Date Range: 2020-01-10 00:00:00 to 2025-01-09 00:00:00 check is False
Processed and saved data for 3MINDIA.


[*********************100%***********************]  1 of 1 completed


...

Reference Dates: (Timestamp('2020-01-10 00:00:00'), Timestamp('2025-01-09 00:00:00')) Current Date Range: 2021-07-23 00:00:00 to 2025-01-09 00:00:00 check is True
Processed and saved data for ZOMATO.


[*********************100%***********************]  1 of 1 completed


Reference Dates: (Timestamp('2020-01-10 00:00:00'), Timestamp('2025-01-09 00:00:00')) Current Date Range: 2020-01-10 00:00:00 to 2025-01-09 00:00:00 check is False
Processed and saved data for ZYDUSLIFE.


[*********************100%***********************]  1 of 1 completed

Reference Dates: (Timestamp('2020-01-10 00:00:00'), Timestamp('2025-01-09 00:00:00')) Current Date Range: 2020-01-10 00:00:00 to 2025-01-09 00:00:00 check is False
Processed and saved data for ECLERX.
All data processing completed.


In [ ]:
import os
import pandas as pd

# Path to the stock data directory
stock_data_directory = "stock_data"

# Dictionary to store processed stock data
NIFTY500_stock = {}

# Function to compute additional features
def compute_features(df):
    # Ensure the DataFrame is sorted by date
    df = df.sort_values(by="Date")
    
    # Convert Date to datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Calculate daily return
    df['return_ratio'] = df['Close'].pct_change()
    
    # Calculate moving averages (e.g., 20-day and 50-day)
    df['MA_20'] = df['Close'].rolling(window=20).mean()
    df['MA_50'] = df['Close'].rolling(window=50).mean()
    
    # Calculate volatility (e.g., 20-day rolling standard deviation of returns)
    df['Volatility_20'] = df['return_ratio'].rolling(window=20).std()
    
    # Drop NA values introduced by rolling computations
    df = df.dropna()
    return df

# Iterate over all CSV files in the stock_data directory
for filename in os.listdir(stock_data_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(stock_data_directory, filename)
        
        # Extract company name (remove .csv extension)
        company_name = filename.replace(".csv", "")
        
        # Load the stock data
        stock_data = pd.read_csv(file_path)
        
        # Compute additional features
        processed_data = compute_features(stock_data)
        
        # Save the processed data back to the CSV file
        processed_data.to_csv(file_path, index=False)
        
        # Store data in the dictionary structure
        NIFTY500_stock[company_name] = {
            "stock_price": processed_data
        }
        
        print(f"Processed and saved: {filename}")

# Access Example
print("Sample Access Example:")
example_company = list(NIFTY500_stock.keys())[0]  # Get the first company
print(f"Company: {example_company}")
print(NIFTY500_stock[example_company]["stock_price"].head())

print("Feature computation and saving completed for all files.")
